In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import time

In [ ]:
train = pd.read_csv('/kaggle/input/shopee-product-matching/train.csv')
train.head()

In [ ]:
test = pd.read_csv('/kaggle/input/shopee-product-matching/test.csv')
test.head()

In [ ]:
print(np.shape(train))
print(f"Unique phashes: {len(np.unique(train['image_phash']))}")
print(f"Unique images: {len(np.unique(train['image']))}")
print(f"Unique titles: {len(np.unique(train['title']))}")

train.info()

We can see that there are no null values in the training set and all columns are objects except the label_group. We also see that there are duplicate values for phashes, images, and titles, which could make matching some listings easier.

# Displaying Images in the Same Group

In [ ]:
train['label_group'].value_counts()

In [ ]:
same_group = train.iloc[np.where(train['label_group'] == 3627744656)]
same_group.head()

In [ ]:
for title in same_group['title'].head():
    print(title)

This notebook is helpful to display images: https://www.kaggle.com/maksymshkliarevskyi/shopee-before-we-start-eda-phash-baseline#Shopee-Price-Match-Guarantee:-Before-we-start

In [ ]:
# from https://www.kaggle.com/maksymshkliarevskyi/shopee-before-we-start-eda-phash-baseline#Shopee-Price-Match-Guarantee:-Before-we-start
import cv2, os
import skimage.io as io
import matplotlib.pyplot as plt
from PIL import Image

def image_viz(image_path):
    """
    Function for visualization.
    Takes path to image as input.
    """
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)    
    plt.imshow(img)
    plt.axis('off')
    
plt.figure(figsize = (15, 10))
train_image_path = '../input/shopee-product-matching/train_images/'
for idx, i in enumerate(same_group['image'].head()):
    plt.subplot(1, 5, idx + 1)
    image_viz(train_image_path + i)
plt.show()

In [ ]:
print(np.shape(same_group))
print(f"Unique phashes: {len(np.unique(same_group['image_phash']))}")
print(f"Unique images: {len(np.unique(same_group['image']))}")
print(f"Unique titles: {len(np.unique(same_group['title']))}")

In [ ]:
print(same_group['image_phash'])

We can see here how 5 listings that are all the same product can all have significantly different titles and images. We can also see how spacing in the title can affect recognizability, since all five listings have the words 'Implora', 'cheek', and 'liptint', but some have spaces and others do not. Of the 51 images in this group, we see there are 31 unique phashes, 44 unique images, and 49 unique titles, which tells us that very few items in this group share an image or title.

In [ ]:
similar_phash_ind = [32164, 16936, 19345]

plt.figure(figsize = (15, 10))
train_image_path = '../input/shopee-product-matching/train_images/'
for idx, img in enumerate(similar_phash_ind):
    print(same_group['image_phash'][img])
    plt.subplot(1, 3, idx + 1)
    image_viz(train_image_path + same_group['image'][img])
plt.show()

Here we can see that all of these images have slightly different phashes but look almost the same. Maybe we can work with this!

# Working with Titles

This notebook seems helpful, with the section about pre-processing titles: https://www.kaggle.com/ruchi798/shopee-eda-rapids-preprocessing?scriptVersionId=58861191